# Kneser-Ney smoothingによるn-gram言語モデルを使った句点挿入(評価)

## [KenLM](https://github.com/kpu/kenlm) で学習済みのモデル(*.klm)を用意

In [ ]:
import kenlm
LM = 'data/3gram.100K.klm'
model = kenlm.Model(LM)
print('{0}-gram model'.format(model.order))

## KenLM API デモ

In [ ]:
sentence = '今日 も いい 天気 です 。 明日 の 天気 も 晴れ らしい 。'
print(model.score(sentence))
def score(s):
    return sum(prob for prob, _, _ in model.full_scores(s))
# Check that total full score = direct score
assert (abs(score(sentence) - model.score(sentence)) < 1e-3)

In [ ]:
def full_scores(sentence):
    # Show scores and n-gram matches
    words = ['<s>'] + sentence.split() + ['</s>']
    full_score = 0.
    for i, (prob, length, oov) in enumerate(model.full_scores(sentence)):
        ngram = words[i+2-length:i+2]
        words_s = ' '.join(ngram)
        print('{0} {1}: {2}'.format(prob, length, words_s))
        if oov:
            print('\t"{0}" is an OOV'.format(words[i+1]))
        full_score+=prob
    print(full_score)
full_scores(sentence)

# 句読点挿入モデル
## 各位置で句点を挿入する/しないで文全体の尤度比較し、尤度が上がれば句点挿入

In [ ]:
def punkt_insert(sentence, punkt='。', window=3, debug=False):
    words = sentence.split()
    pnkt_pos = []
    for i in range(len(words)-window+1):
        prob = model.score(' '.join(words))
        prob_p = model.score(' '.join(words[:i+window] + [punkt] + words[i+window:]))
        if prob_p > prob:
            if debug:
                print(prob_p, prob, words[i:i+window])
            pnkt_pos.append(i+window)
    return pnkt_pos

def punkt_decode(sentence, punkt_pos, punkt='。'):
    words = sentence.split()
    pnkt_sent = []
    for i in range(len(words)):
        pnkt_sent.append(words[i])
        if i+1 in punkt_pos:
            pnkt_sent.append(punkt)
    return ' '.join(pnkt_sent)

def test_decode(sentence, punkt='。'):
    ins = punkt_decode(sentence, punkt_insert(sentence, punkt=punkt, debug=True), punkt=punkt)
    print(ins)

### 句点挿入による尤度変化の可視化

In [ ]:
sentence_raw = '競馬 の 業務 ・ 運営 を 行う 目的 と 関係 者 や 観客 が 観戦 を する 目的 を 兼ね備え た 施設 スタンド 内 に は 観客 が 立ち入る こと が できる スペース 馬主 など の 関係 者 が 立ち入る こと が 出来る スペース 運営 スタッフ のみ が 立ち入る こと が 出来る スペース が 明確 に 区切ら れ て いる それぞれ 自分 に 関係 し ない スペース へ の 立ち入り は 堅く 禁じ られ て いる 日本 で は 走路 の 決勝 線 の ある 側 の 外周 に 作ら れる こと が 多い 以下 で は スタンド 内 に 併設 さ れ て いる 設備 や 装置 を 説明 する 決勝 線 の 延長線 上 に 設け られ て おり 写真 判定 に 用いる 写真 を ゴール 板 を 利用 し て 撮影 する 中 は レース が 始まる と 真っ暗 闇 に なり 決勝 写真 を 撮り 終え たら すぐさま 手探り で フィルム を 現像 し 下 の 階 の 審判 室 に エレベーター で 送ら れる 勝馬 投票 券 （ 馬券 ） の 販売 ならびに 払い戻し を 行う 設備 かつて は 有人 の 発券 ・ 払い戻し 窓口 が ずらりと 並ん で い た が 現在 は 人件 費 削減 ・ 省力 化 の ため に 機械 化 さ れ て いる 窓口 が 多い なお 一部 は 払い戻し も 兼ね た 兼用 機 に なっ て いる 場合 も ある 対面 式 の 有人 窓口 も ある が これ について も マークシート 使用 による 窓口 と なっ て いる 場合 が 多い 口頭 による 窓口 販売 は ごく 一部 の 窓口 に 限ら れ て いる 一部 に マークカード 対応 窓口 しか ない 競馬 場 も ある が この 場合 でも バリアフリー の 観点 など から マークカード を 扱え ない 障害 者 や 高齢 者 の 馬券 購入 希望 が ある 場合 これ に 応じ て 特定 の 有人 窓口 の 端末 を 手動 操作 する こと など で 発券 対応 し て いる'

In [ ]:
test_decode(sentence_raw, '。')

In [ ]:
sentence_true = '競馬 の 業務 ・ 運営 を 行う 目的 と 関係 者 や 観客 が 観戦 を する 目的 を 兼ね備え た 施設 。 スタンド 内 に は 観客 が 立ち入る こと が できる スペース 、 馬主 など の 関係 者 が 立ち入る こと が 出来る スペース 、 運営 スタッフ のみ が 立ち入る こと が 出来る スペース が 明確 に 区切ら れ て いる 。 それぞれ 自分 に 関係 し ない スペース へ の 立ち入り は 堅く 禁じ られ て いる 。 日本 で は 走路 の 決勝 線 の ある 側 の 外周 に 作ら れる こと が 多い 。 以下 で は スタンド 内 に 併設 さ れ て いる 設備 や 装置 を 説明 する 。 決勝 線 の 延長線 上 に 設け られ て おり 、 写真 判定 に 用いる 写真 を ゴール 板 を 利用 し て 撮影 する 。 中 は レース が 始まる と 真っ暗 闇 に なり 決勝 写真 を 撮り 終え たら すぐさま 手探り で フィルム を 現像 し 、 下 の 階 の 審判 室 に エレベーター で 送ら れる 。 勝馬 投票 券 （ 馬券 ） の 販売 ならびに 払い戻し を 行う 設備 。 かつて は 有人 の 発券 ・ 払い戻し 窓口 が ずらりと 並ん で い た が 、 現在 は 人件 費 削減 ・ 省力 化 の ため に 機械 化 さ れ て いる 窓口 が 多い 。 なお 、 一部 は 払い戻し も 兼ね た 兼用 機 に なっ て いる 場合 も ある 。 対面 式 の 有人 窓口 も ある が これ について も マークシート 使用 による 窓口 と なっ て いる 場合 が 多い 。 口頭 による 窓口 販売 は ごく 一部 の 窓口 に 限ら れ て いる 。 一部 に マークカード 対応 窓口 しか ない 競馬 場 も ある が 、 この 場合 でも バリアフリー の 観点 など から マークカード を 扱え ない 障害 者 や 高齢 者 の 馬券 購入 希望 が ある 場合 、 これ に 応じ て 特定 の 有人 窓口 の 端末 を 手動 操作 する こと など で 発券 対応 し て いる 。'

# 評価
## 句読点付き文に対して、句点抜き文への句点挿入の精度と再現率を評価
### 現状、句点か読点かいずれか一方(punkt)の挿入のみ実装

In [ ]:
# 句読点付き文から句点挿入位置を抽出
def extract_positions(sentence, punkt='。'):
    PUNKTS = ['、', '。']
    true_positions = []
    c_pos = 0
    for w in sentence.split(' '):
        if w in PUNKTS:
            if w == punkt:
                true_positions.append(c_pos)
        else:
            c_pos += 1
    return true_positions

In [ ]:
# 句読点無し文に対する句点挿入位置の予測結果から精度と再現率を計算
def prf(punkt_pred, punkt_true):
    tp = set.intersection(punkt_pred, punkt_true)      
    precision = len(tp) / len(punkt_true) if len(punkt_true) > 0 else 0
    recall = len(tp) / len(punkt_pred) if len(punkt_pred) > 0 else 0
    fval = 2 / (1/precision + 1/recall) if precision and recall else 0.
    return precision, recall, fval

# 句点未挿入の文に対して精度・再現率を計算
def evaluate(sentence_raw, sentence_true, punkt='。'):
    punkt_pred = set(punkt_insert(sentence_raw, punkt))
    punkt_true = set(extract_positions(sentence_true, punkt))
    return prf(punkt_pred, punkt_true)

# 句点挿入済み結果の文に対して精度・再現率を計算
def evaluate_sym(sentence_pred, sentence_true, punkt='。'):
    punkt_pred = set(extract_positions(sentence_pred, punkt))
    punkt_true = set(extract_positions(sentence_true, punkt))
    return prf(punkt_pred, punkt_true)

In [ ]:
# 上の文について計算
evaluate(sentence_raw, sentence_true, punkt='。')

## 評価単体テスト

### 低Recallの例

In [ ]:
sentence_raw = 'この 場合 でも バリアフリー の 観点 など から マークカード を 扱え ない 障害 者 や 高齢 者 の 馬券 購入 希望 が ある 場合 これ に 応じ て 特定 の 有人 窓口 の 端末 を 手動 操作 する こと など で 発券 対応 し て いる'
sentence_true = 'この 場合 でも バリアフリー の 観点 など から マークカード を 扱え ない 障害 者 や 高齢 者 の 馬券 購入 希望 が ある 場合 、 これ に 応じ て 特定 の 有人 窓口 の 端末 を 手動 操作 する こと など で 発券 対応 し て いる 。'

In [ ]:
# 挿入位置
punkt_insert(sentence_raw, punkt='。', debug=True)

In [ ]:
# 正解位置
extract_positions(sentence_true, punkt='。')

In [ ]:
# P/R/F計算
evaluate(sentence_raw, sentence_true, punkt='。')

### 低Precisionの例

In [ ]:
sentence_raw = '今日 も いい 天気 明日 の 天気 も 晴れ らしい'
sentence_true = '今日 も 、 いい 天気 。 明日 の 天気 も 晴れ らしい 。' 

In [ ]:
# 挿入位置
punkt_insert(sentence_raw, punkt='。', debug=True)

In [ ]:
# 正解位置
extract_positions(sentence_true, punkt='。')

In [ ]:
# P/R/F計算
evaluate(sentence_raw, sentence_true, punkt='。')

# テストデータによる評価
## 未知語置換済み / 適当にカット済みデータに対して評価
### 比較的まともに終了するサイズとして、約10文ごとの文ブロックを1万ブロック

In [ ]:
from itertools import chain

# 頻度上位100,000以下は未知語記号<UNK>で置換
vocab_path = 'data/train.txt.100K' # 学習に使ったデータセット
target_path = 'data/target_test.txt'
source_path = 'data/source_test.txt'

with open(vocab_path) as fp:
    vocab = set(chain.from_iterable([l.split(' ') for l in fp.read().splitlines()]))

with open(target_path) as tfp, open(source_path) as sfp, \
     open(target_path+'.100K', 'w') as tofp, \
     open(source_path+'.100K', 'w') as sofp:
    for tl, sl in zip(tfp,sfp):
        words_to_write = []
        for w in tl.strip().split(' '):
            if w in vocab:
                words_to_write.append(w)
            elif '、' in w:
                words_to_write.append('、')
            elif '。' in w:
                words_to_write.append('。')
            else:
                words_to_write.append('<UNK>')
        tofp.write(' '.join(words_to_write)+'\n')
        words_to_write = []
        for w in sl.strip().split(' '):
            if w in vocab:
                words_to_write.append(w)
            else:
                words_to_write.append('<UNK>')
        sofp.write(' '.join(words_to_write)+'\n')


In [ ]:
BOUND = 10000
with open('data/source_test.txt.100K') as fp:
    source_test = fp.read().splitlines()[:BOUND]
with open('data/target_test.txt.100K') as fp:
    target_test = fp.read().splitlines()[:BOUND]

In [ ]:
micros = []
with open('data/prf.100K.{}'.format(BOUND), 'w') as ofp:
    for sent_raw, sent_true in zip(source_test, target_test):
        if len(extract_positions(sent_true)) > 0:
            p, r, f = evaluate(sent_raw, sent_true)
            ofp.write('{0:5g} {1:5g} {2:5g}\n'.format(p,r,f))
            micros.append((p,r,f))

In [ ]:
import numpy as np
micros = np.array(micros)
print(micros.mean(axis=0), len(micros))

### 上までで十分だが、エラー分析のため一旦句点挿入したのちPRF値評価もする場合も評価

In [ ]:
with open('data/source_test.txt.100K') as fp, open('data/decoded.100K.{}'.format(BOUND), 'w') as ofp:
    for sent_raw in fp.read().splitlines()[:BOUND]:
        ofp.write(punkt_decode(sent_raw, punkt_insert(sent_raw.strip()))+'\n')

In [ ]:
with open('data/decoded.100K.{}'.format(BOUND)) as fp:
    decoded = fp.read().splitlines()
with open('data/target_test.txt.100K') as fp:
    target_test = fp.read().splitlines()[:BOUND]
print('\n'.join(decoded[:3]))
print('\n'.join(target_test[:3]))
for sent_raw, sent_true in zip(decoded[:5], target_test[:5]):
    print(evaluate_sym(sent_raw, sent_true))

In [ ]:
with open('data/decoded.100K.{}'.format(BOUND)) as fp:
    decoded = fp.read().splitlines()
with open('data/target_test.txt.100K') as fp:
    target_test = fp.read().splitlines()[:BOUND]
micros = []
with open('data/prf.100K.{}.d'.format(BOUND), 'w') as ofp:
    for sent_raw, sent_true in zip(decoded, target_test):
        p, r, f = evaluate_sym(sent_raw, sent_true)
        ofp.write('{0:5g} {1:5g} {2:5g}\n'.format(p,r,f))
        micros.append((p,r,f))

In [ ]:
import numpy as np
micros = np.array(micros)
print(micros.mean(axis=0), micros.shape[0])